In [1]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

In [10]:

#Instance (n-s-m-l-x)
# model = YOLO('yolov8s-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8s-seg.pt')  # Transfer the weights from a pretrained model (recommended for training)
folder = "C:/Users/domin/Leaf Color Detector/LCD24/YOLO/dataset-per-week/yolo_dataset_w1"
#Define subdirectory for this specific training
modelname = "test-10-epoch" #note that if you run the training again, it creates a directory: 200_epochs-2
# name = "100_epochs_s_w1-" #note that if you run the training again, it creates a directory: 200_epochs-2
epochs = 10

In [11]:
# define number of classes based on YAML
import yaml
with open(folder + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#Define a project --> Destination directory for all results
project = folder + "/results"

#TODO check if the image size has an impact on the training
# Train the model
results = model.train(data= folder + "/data.yaml",
                      project=project,
                      name=modelname,
                      epochs=epochs,
                      patience=0, #I am setting patience=0 to disable early stopping.
                      batch=4,
                      imgsz=500,
                      mosaic = 0.0, #disables mosaic augmentation
                      scale = 0 #disables multi-scale training
                      )

In [ ]:
from IPython.display import Image
Image(filename=folder + '/results/'+modelname+'/results.png')  # View the training image



### Inference

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
mynewmodel = YOLO(folder + '/results/' + modelname + '/weights/best.pt')  # Load the best model from the 'train' subdirectory
#results = mynewmodel.test(data="C:/Users/domin/Leaf Color Detector/YOLO/small-dataset/yolo_dataset/data.yaml", batch_size=4)  # Test the best model
new_image = 'C:/Users/domin/Leaf Color Detector/Leaf_Color_Detector/spinach/no overlap/all pictures/108_w1.jpg'
new_results = mynewmodel.predict(new_image, conf = 0.8)  # Inference on a single image

new_result_array = new_results[0].plot()
plt.figure(figsize=(12, 12))
plt.imshow(new_result_array)

In [ ]:
import os

# Define the input and output directories
input_dir = "C:/Users/domin/Leaf Color Detector/Leaf_Color_Detector/spinach/no overlap/all pictures"
output_dir = "C:/Users/domin/Leaf Color Detector/yolo results/100 epochs s"
mynewmodel = YOLO('C:/Users/domin/Leaf Color Detector/LCD24/YOLO/dataset/yolo_dataset/results/100_epochs_s-/weights/best.pt')  # Load the best model from the 'train' subdirectory

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over the files in the input directory
for filename in os.listdir(input_dir):
    if "w" in filename:
        # Construct the full path of the image
        image_path = os.path.join(input_dir, filename)
        
        # Perform inference on the image
        results = mynewmodel.predict(image_path, conf=0.7)
        result_array = results[0].plot()
        
        # Save the plot in the output directory
        output_path = os.path.join(output_dir, f"{filename.split('.')[0]}.png")
        plt.figure(figsize=(12, 12))
        plt.imshow(result_array)
        plt.savefig(output_path)
        plt.close()